In [1]:
!pip install mediapipe opencv-python matplotlib


In [1]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt

print("Mediapipe version:", mp.__version__)


Mediapipe version: 0.10.21


In [13]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# EAR calculation using selected eye landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

def euclidean_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def eye_aspect_ratio(eye_points, landmarks):
    # vertical distances
    A = euclidean_distance(landmarks[eye_points[1]], landmarks[eye_points[5]])
    B = euclidean_distance(landmarks[eye_points[2]], landmarks[eye_points[4]])
    # horizontal distance
    C = euclidean_distance(landmarks[eye_points[0]], landmarks[eye_points[3]])
    # EAR
    return (A + B) / (2.0 * C)

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert color
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    sleep_status = "Face not detected"

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            landmarks = [(int(p.x * w), int(p.y * h)) for p in face_landmarks.landmark]

            left_ear = eye_aspect_ratio(LEFT_EYE, landmarks)
            right_ear = eye_aspect_ratio(RIGHT_EYE, landmarks)
            avg_ear = (left_ear + right_ear) / 2.0

            # Determine sleep/wake based on EAR threshold
            if avg_ear < 0.21:
                sleep_status = "Sleeping 😴"
            else:
                sleep_status = "Awake 😄"

            # Draw landmarks
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

    # Display status
    cv2.putText(frame, f'Status: {sleep_status}', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 0), 3)
    cv2.imshow("Sleep Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
